1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [ ]:
pip install implicit==0.4.8

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606575 sha256=e34fae89466743cac5cce2950ea519426e518607f299e1eceb8a7c82cc39edc7
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [ ]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

class MainRecommender:
  """ Рекомендаци, которые можно получить из ALS

  Input
  -----
  user item_matrix: pd.DataFrame
    Матрица взаимодействий user-item
  """

  def __init__(self, data, weighting=True):
    # your_code (не обязательно)
    self.user_item_matrix = self.prepare_matrix(data) # pd.DataFrame
    self.id_to_itemid, self.id_to_itemid,self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)

    if weighting:
      self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T
    
    self.model = self.fit(self.user_item_matrix)
    self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

  @staticmethod
  def prepare_matrix(data):
    
    # your code
    user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )
    user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit


    return user_item_matrix

  @staticmethod
  def prepare_dicts(user_item_matrix):
    """ Подготавливает вспомогательные словари """

    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values

    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))

    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))

    return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

  @staticmethod
  def fit_own_recommender(user_item_matrix):
    """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

    own_recommender = ItemItemRecommender(K=1, num_threads=4)
    own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

    return own_recommender

  @staticmethod
  def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
    """Обучает ALS"""

    model = AlternatingLeasSquares(factors=factors, regularization=regularization,
                                   iterations=iterations,
                                   num_threads=num_threads)
    model.fit(csr_matrix(self.user_item_matrix).T.tocsr())

    return model

  def get_simular_users_recommendation(self, user, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

    # your_code
    # Практически полностью реализовали на прошлом вебинаре
    top_user_items = user_item_matrix.loc[self.user_item_matrix[user_id] == user].head(N)
    for id_item in top_user_items:
      item = model.similar_items(id=userid_to_id[user], 1)
      res.append(item)
                                    

    assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
    return res

  def get_simular_users_recommendation(self, user, N=5):
    """ Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    model.similar_users(self.userid_to_id[user], N=5)
    # your code
    assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
    return res

In [ ]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=True)]
    return res

In [ ]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товарыs
    
    # ...